In [194]:
import sympy
from beartype.typing import List
from beartype import beartype
from numbers import Real


@beartype
class BezierCurve:
    """
    (see)[https://en.wikipedia.org/wiki/B%C3%A9zier_curve]
    """

    def __init__(self, control_points : (sympy.Matrix, sympy.MatrixSymbol), t_start: Real, t_stop: Real):
        """
        @param control_points: sympy matrix with horiz stacked control points (vertical vectors)
        @param t_start : start time in seconds
        @param t_stop : stop time in seconds
        """
        self.control_points = control_points
        self.m = self.control_points.shape[0]
        self.n = self.control_points.shape[1]-1
        self.t_start = t_start
        self.t_stop = t_stop
    
    def eval(self, t : (Real, sympy.Symbol)):
        """
        (see)[https://en.wikipedia.org/wiki/De_Casteljau%27s_algorithm]
        @param t : time in seconds
        """
        beta = (t - self.t_start)/(self.t_stop - self.t_start)
        A = sympy.Matrix(self.control_points)
        for j in range(1, self.n + 1):
            for k in range(self.n + 1 - j):
                A[:, k] = A[:, k] * (1 - beta) + A[:, k + 1] * beta
        return A[:, 0]
    
    def deriv(self, order : int = 1):
        """
        @param order: order of derivative
        """
        D = sympy.Matrix(self.control_points)
        for j in range(0, order):
            D = (self.n - j)*sympy.Matrix.hstack(*[ D[:, i+1] - D[:, i] for i in range(self.n - j) ])
        T = self.t_stop - self.t_start
        return BezierCurve(control_points=D/T**order, t_start=self.t_start, t_stop=self.t_stop)

    def _repr_latex_(self):
        return self.control_points._repr_latex_()


In [232]:
t = sympy.symbols('t')
P = sympy.Matrix([[0, 1, 3],
                  [0, 2, 4],
                  [3, 2, 1]
                 ])
P = sympy.MatrixSymbol('P', 3, 5)
B = BezierCurve(control_points=P, t_start=0, t_stop=1)

B_d = B.deriv()
B_d2 = B_d.deriv()
B_d3 = B_d2.deriv()
B_d4 = B_d3.deriv()

In [234]:
constraints = []
constraints += [(B.eval(0), wp_0[0])]

NameError: name 'wp_0' is not defined

In [231]:
B_d

In [228]:
p = B.eval(t).applyfunc(lambda x: x.simplify().collect(t))
p

Matrix([
[t**4*(P[0, 0] - 4*P[0, 1] + 6*P[0, 2] - 4*P[0, 3] + P[0, 4]) + t**3*(-4*P[0, 0] + 12*P[0, 1] - 12*P[0, 2] + 4*P[0, 3]) + t**2*(6*P[0, 0] - 12*P[0, 1] + 6*P[0, 2]) + t*(-4*P[0, 0] + 4*P[0, 1]) + P[0, 0]],
[t**4*(P[1, 0] - 4*P[1, 1] + 6*P[1, 2] - 4*P[1, 3] + P[1, 4]) + t**3*(-4*P[1, 0] + 12*P[1, 1] - 12*P[1, 2] + 4*P[1, 3]) + t**2*(6*P[1, 0] - 12*P[1, 1] + 6*P[1, 2]) + t*(-4*P[1, 0] + 4*P[1, 1]) + P[1, 0]],
[t**4*(P[2, 0] - 4*P[2, 1] + 6*P[2, 2] - 4*P[2, 3] + P[2, 4]) + t**3*(-4*P[2, 0] + 12*P[2, 1] - 12*P[2, 2] + 4*P[2, 3]) + t**2*(6*P[2, 0] - 12*P[2, 1] + 6*P[2, 2]) + t*(-4*P[2, 0] + 4*P[2, 1]) + P[2, 0]]])

In [229]:
v = p.diff(t).applyfunc(lambda x: x.simplify().collect(t))
v

Matrix([
[t**3*(4*P[0, 0] - 16*P[0, 1] + 24*P[0, 2] - 16*P[0, 3] + 4*P[0, 4]) + t**2*(-12*P[0, 0] + 36*P[0, 1] - 36*P[0, 2] + 12*P[0, 3]) + t*(12*P[0, 0] - 24*P[0, 1] + 12*P[0, 2]) - 4*P[0, 0] + 4*P[0, 1]],
[t**3*(4*P[1, 0] - 16*P[1, 1] + 24*P[1, 2] - 16*P[1, 3] + 4*P[1, 4]) + t**2*(-12*P[1, 0] + 36*P[1, 1] - 36*P[1, 2] + 12*P[1, 3]) + t*(12*P[1, 0] - 24*P[1, 1] + 12*P[1, 2]) - 4*P[1, 0] + 4*P[1, 1]],
[t**3*(4*P[2, 0] - 16*P[2, 1] + 24*P[2, 2] - 16*P[2, 3] + 4*P[2, 4]) + t**2*(-12*P[2, 0] + 36*P[2, 1] - 36*P[2, 2] + 12*P[2, 3]) + t*(12*P[2, 0] - 24*P[2, 1] + 12*P[2, 2]) - 4*P[2, 0] + 4*P[2, 1]]])

In [230]:
a = v.diff(t).applyfunc(lambda x: x.simplify().collect(t))
a

Matrix([
[t**2*(12*P[0, 0] - 48*P[0, 1] + 72*P[0, 2] - 48*P[0, 3] + 12*P[0, 4]) + t*(-24*P[0, 0] + 72*P[0, 1] - 72*P[0, 2] + 24*P[0, 3]) + 12*P[0, 0] - 24*P[0, 1] + 12*P[0, 2]],
[t**2*(12*P[1, 0] - 48*P[1, 1] + 72*P[1, 2] - 48*P[1, 3] + 12*P[1, 4]) + t*(-24*P[1, 0] + 72*P[1, 1] - 72*P[1, 2] + 24*P[1, 3]) + 12*P[1, 0] - 24*P[1, 1] + 12*P[1, 2]],
[t**2*(12*P[2, 0] - 48*P[2, 1] + 72*P[2, 2] - 48*P[2, 3] + 12*P[2, 4]) + t*(-24*P[2, 0] + 72*P[2, 1] - 72*P[2, 2] + 24*P[2, 3]) + 12*P[2, 0] - 24*P[2, 1] + 12*P[2, 2]]])

In [189]:
bc.deriv(order=1).eval(t)

Matrix([
[2*t + 2],
[      4],
[     -2]])